# Tarea 11 - SimPy: Venta de boletos 2D/3D

> Asignatura: Simulación<br/>
> Estudiante: Eduardo Zhizhpon

Al modelo de simulación de la venta de boletos de un teatro/cine agregar la simulación grafica 2D/3D utilizando cualquier libreria grafica, por ejemplo pygame/tkinter, etc. generar un informe en formato PDF que me permita evidenciar la simulación y generar una grafica/reporte de los resultados.

In [5]:
import collections
import random
import simpy

VELOCIDAD_LLEGADA = 42
NUM_BOLETO = 50
TIEMPO_SIMULACION = 120

def ventaBoletos(env, num_boletos, pelicula, teatro):
    with teatro.contador.request() as turno:
        resultado = yield turno | teatro.sold_out[pelicula]
        if turno not in resultado:
            teatro.num_renegados[pelicula] += 1
            return
        if teatro.num_boletos[pelicula] < num_boletos:
            yield env.timeout(0.5) 
            return
        teatro.num_boletos[pelicula] -= num_boletos
        if teatro.num_boletos[pelicula] < 2:
            teatro.sold_out[pelicula].succeed()
            teatro.tiempo_agotado[pelicula] = env.now
            teatro.num_boletos[pelicula] = 0
        yield env.timeout(1)


def llegadaClientes(env, teatro):
    while True:
        yield env.timeout(random.expovariate(1/0.5))
        pelicula = random.choices(teatro.peliculas, teatro.probabilidad, k=1)
        num_boletos = random.randint(1, 6)
        if teatro.num_boletos[pelicula[0]]:
            env.process(ventaBoletos(env, num_boletos, pelicula[0], teatro))

In [4]:
Teatro = collections.namedtuple('Teatro', 'contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
peliculas = ['Conjuro 3', 'Rapidos y Furiosos 10', 'Pulp Fictions']
probabilidad=[0.1, 0.3, 0.6]
num_boletos = {pelicula: NUM_BOLETO for pelicula in peliculas}
sold_out = {pelicula: env.event() for pelicula in peliculas}
tiempo_agotado = {pelicula: None for pelicula in peliculas}
num_renegados = {pelicula: 0 for pelicula in peliculas}

teatro = Teatro(contador, peliculas, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, teatro))
env.run(until=TIEMPO_SIMULACION)

# Analisis y resultados
for pelicula in peliculas:
    if teatro.sold_out[pelicula]:
        print('Pelicula: %s se agoto en el tiempo %.1f despues de salir a la venta' %(pelicula, teatro.tiempo_agotado[pelicula]))
        print('Numero de personas que salieron de la fila/renegados %s' %teatro.num_renegados[pelicula])

Teatro Carlos Crespi - UPS
Pelicula: Conjuro 3 se agoto en el tiempo 74.4 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Rapidos y Furiosos 10 se agoto en el tiempo 35.9 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Pelicula: Pulp Fictions se agoto en el tiempo 16.5 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 9


In [6]:
!pip install pygame

     |████████████████████████████████| 21.9 MB 6.3 MB/s eta 0:00:01


In [7]:
import pygame
import sys

size = (800, 500)

screen = pygame.display.set_mode(size)

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/opt/miniconda3/envs/p60/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
